# Mount Drive

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


# Install Library

In [3]:
!pip install pyarrow==15.0.0

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 38.3/38.3 MB 16.2 MB/s eta 0:00:00
  Attempting uninstall: pyarrow
    Found existing installation: pyarrow 18.1.0
    Uninstalling pyarrow-18.1.0:
      Successfully uninstalled pyarrow-18.1.0


In [4]:
!pip install datasets sentencepiece --quiet
# !pip install transformers --quiet
# !pip install accelerate --quiet
!pip install transformers[torch] --quiet
!pip install peft --quiet
!pip install evaluate --quiet
!pip install -U bitsandbytes --quiet

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 487.4/487.4 kB 11.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 5.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 4.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.8/194.8 kB 6.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 54.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 35.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 30.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 1.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 2.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 15.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [5]:
import transformers
import numpy as np
import json
import os
from torch.utils.data import Dataset, random_split
import torch
from peft import get_peft_model, LoraConfig, TaskType

In [ ]:
from transformers import BertTokenizerFast
tokenizer = BertTokenizerFast.from_pretrained("bert-base-uncased")

tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

In [ ]:
if torch.cuda.is_available():
    print("CUDA is available. Using GPU.")
    device = torch.device("cuda")
else:
    print("CUDA not available. Using CPU.")
    device = torch.device("cpu")

CUDA is available. Using GPU.


# Load Pre-train Model

In [ ]:
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM,  MT5ForConditionalGeneration, AutoModelForCausalLM, BitsAndBytesConfig, MBartTokenizer, MBartForConditionalGeneration
pretrain_model_name = "facebook/mbart-large-50"

tokenizer = MBartTokenizer.from_pretrained(pretrain_model_name)

tokenizer_config.json:   0%|          | 0.00/531 [00:00<?, ?B/s]

sentencepiece.bpe.model:   0%|          | 0.00/5.07M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/649 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.42k [00:00<?, ?B/s]

The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'MBart50Tokenizer'. 
The class this function is called from is 'MBartTokenizer'.


In [ ]:
SPECIAL_TOKEN = ['<SYMBOL>','<ASPECT>','<OPINION>','<POS>','<NEG>','<NEU>']
special_tokens_dict = {'additional_special_tokens': SPECIAL_TOKEN}
num_added_tokens = tokenizer.add_special_tokens(special_tokens_dict)

In [ ]:
# bitsandbytes parameters
################################################################################

# Activate 4-bit precision base model loading
use_4bit = True

# Compute dtype for 4-bit base models
bnb_4bit_compute_dtype = "float16"

# Quantization type (fp4 or nf4)
bnb_4bit_quant_type = "nf4"

# Activate nested quantization for 4-bit base models (double quantization)
use_nested_quant = False


In [ ]:
# Load tokenizer and model with QLoRA configuration
# AI model normally 32 bits too big --> need quantile to reduce size - to be able to run the model but trade off with accuracy
compute_dtype = getattr(torch, bnb_4bit_compute_dtype)

bnb_config = BitsAndBytesConfig(
    load_in_4bit=use_4bit,
    bnb_4bit_quant_type=bnb_4bit_quant_type,
    bnb_4bit_compute_dtype=compute_dtype,
    bnb_4bit_use_double_quant=use_nested_quant,
)

In [ ]:
# Define LoRA Config
lora_config = LoraConfig(
    r=16,  # Rank of the LoRA update matrices
    lora_alpha=32,  # Scaling factor for the LoRA update matrices
    lora_dropout=0.05,  # Dropout probability for the LoRA update matrices
    bias="none",  # Bias type for the LoRA update matrices
    task_type=TaskType.CAUSAL_LM,  # Task type for the LoRA update matrices
)

In [ ]:
model = MBartForConditionalGeneration.from_pretrained(pretrain_model_name)
model.resize_token_embeddings(len(tokenizer))

# Wrap the model with PEFT
#model = get_peft_model(model, lora_config)

pytorch_model.bin:   0%|          | 0.00/2.44G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/261 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/2.44G [00:00<?, ?B/s]

The new embeddings will be initialized from a multivariate normal distribution that has old embeddings' mean and covariance. As described in this article: https://nlp.stanford.edu/~johnhew/vocab-expansion.html. To disable this, use `mean_resizing=False`


MBartScaledWordEmbedding(250060, 1024, padding_idx=1)

#Load Dataset

In [ ]:
dataset_folder = '/content/drive/MyDrive/ABSA_Datasets'
dataset_filename = 'Text-to-Label-Dataset.csv'

In [ ]:
from datasets import load_dataset
dataset = load_dataset('csv', data_files=os.path.join(dataset_folder, dataset_filename))
# dataset = load_dataset('csv', data_files={'train': 'train.csv', 'validation': 'valid.csv', 'test': 'test.csv'})

Generating train split: 0 examples [00:00, ? examples/s]

In [ ]:
dataset

DatasetDict({
    train: Dataset({
        features: ['text', 'label'],
        num_rows: 8368
    })
})

In [ ]:
dataset['train'][0]

{'text': 'DOHOME รายได้ขายเพิ่ม! ดันกำไร Q2 โต 46% แตะ 436 ล้านบาท',
 'label': '<SYMBOL> DOHOME<ASPECT> รายได้<OPINION> เพิ่ม<POS>'}

In [ ]:
def process_data_to_model_inputs(batch):
    # tokenize the text and labels
    input_encoding = tokenizer(batch["text"], padding="max_length", truncation=True, max_length=512)
    batch["input_ids"] = input_encoding.input_ids
    batch["attention_mask"] = input_encoding.attention_mask
    output_encoding = tokenizer(batch["label"], padding="max_length", truncation=True, max_length=512)
    batch["labels"] = output_encoding.input_ids
    batch["decoder_attention_mask"] = output_encoding["attention_mask"]
    return batch

# Check if the tokenizer has a padding token. If not, add one.
#if tokenizer.pad_token is None:
    #tokenizer.add_special_tokens({'pad_token': '[PAD]'})

In [ ]:
# Tokenize the dataset
tokenized_dataset = dataset.map(process_data_to_model_inputs, batched=True)
# tokenized_dataset = dataset.map(preprocess_function, batched=True)

Map:   0%|          | 0/8368 [00:00<?, ? examples/s]

In [ ]:
tokenized_dataset

DatasetDict({
    train: Dataset({
        features: ['text', 'label', 'input_ids', 'attention_mask', 'labels', 'decoder_attention_mask'],
        num_rows: 8368
    })
})

In [ ]:
tokenized_dataset['train'].train_test_split(test_size=0.2)

DatasetDict({
    train: Dataset({
        features: ['text', 'label', 'input_ids', 'attention_mask', 'labels', 'decoder_attention_mask'],
        num_rows: 6694
    })
    test: Dataset({
        features: ['text', 'label', 'input_ids', 'attention_mask', 'labels', 'decoder_attention_mask'],
        num_rows: 1674
    })
})

In [ ]:
tokenized_dataset = tokenized_dataset.remove_columns(['text','label'])

In [ ]:
datacollator = lambda data: {
                              # 'input_ids': torch.stack([f[0] for f in data]),
                              'input_ids': data['input_ids'],
                              # 'attention_mask': torch.stack([f[1] for f in data]),
                              # 'labels': torch.stack([f[0] for f in data]),
                              'labels': data['labels'],
                              }

In [ ]:
def split_dataset(dataset, train_size=0.8, test_size=0.1, seed=42):

    # Calculate the validation size as the remaining portion not used by train and test
    val_size = 1 - (train_size + test_size)

    # Ensure the sizes sum up to 1
    if train_size + test_size + val_size != 1:
        raise ValueError("Train, test, and validation sizes must sum up to 1.")

    # Split the dataset into train, test, and validation sets
    dataset = dataset["train"].train_test_split(test_size=test_size + val_size, seed=seed)

    # Further split the test set into validation and test sets
    test_val_dataset = dataset['test'].train_test_split(test_size=val_size / (test_size + val_size), seed=seed)

    return {
        "train": dataset["train"],
        "validation": test_val_dataset["train"],
        "test": test_val_dataset["test"]
    }

In [ ]:
splits = split_dataset(tokenized_dataset)

In [ ]:
splits

{'train': Dataset({
     features: ['input_ids', 'attention_mask', 'labels', 'decoder_attention_mask'],
     num_rows: 6694
 }),
 'validation': Dataset({
     features: ['input_ids', 'attention_mask', 'labels', 'decoder_attention_mask'],
     num_rows: 837
 }),
 'test': Dataset({
     features: ['input_ids', 'attention_mask', 'labels', 'decoder_attention_mask'],
     num_rows: 837
 })}

#Fine-tuning

In [ ]:
project_root = '/content/drive/MyDrive/ABSA_Datasets/Model'
model_folder = 'test_model_attention_mask'
model_path = os.path.join(project_root, model_folder)
os.makedirs(model_path, exist_ok=True)
model_path

'/content/drive/MyDrive/ABSA_Datasets/Model/test_model_attention_mask'

In [ ]:
from transformers import Trainer, TrainingArguments

In [ ]:
# training_args = TrainingArguments(
#     output_dir="./results",          # output directory
#     num_train_epochs=3,              # total number of training epochs
#     per_device_train_batch_size=8,   # batch size per device during training
#     per_device_eval_batch_size=8,    # batch size for evaluation
#     warmup_steps=500,                # number of warmup steps for learning rate scheduler
#     weight_decay=0.01,               # strength of weight decay
#     logging_dir='./logs',            # directory for storing logs
# )

training_args = TrainingArguments(output_dir=os.path.join(model_path,'model_output'),
                                  num_train_epochs=20,
                                  logging_steps=100,
                                  # save_steps=5000,
                                  per_device_train_batch_size=10,
                                  per_device_eval_batch_size=10,
                                  warmup_steps=500,
                                  weight_decay=0.05,
                                  logging_dir=os.path.join(model_path,'logs'),
)

In [ ]:
# Initialize Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=splits["train"],
    eval_dataset=splits["validation"],
    # data_collator=datacollator,
    # data_collector=lambda data: {'input_ids':

    # }
)


In [ ]:
for param in model.parameters():
    param.data = param.data.contiguous()

In [ ]:
# Train the model
trainer.train()

wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.


<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter:

 ··········


wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: teenodie (teenodie-king-mongkut-s-institute-of-technology-ladkrabang) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin
wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.


Step,Training Loss
100,11.751100
200,4.095100
300,0.331200
400,0.023300
500,0.010300
600,0.008300
700,0.007200
800,0.006300
900,0.006600
1000,0.006400


/usr/local/lib/python3.11/dist-packages/transformers/modeling_utils.py:2758: UserWarning: Moving the following attributes in the config to the generation config: {'max_length': 200, 'early_stopping': True, 'num_beams': 5}. You are seeing this warning because you've set generation parameters in the model config, as opposed to in the generation config.
  warnings.warn(


TrainOutput(global_step=13400, training_loss=0.12384197865515502, metrics={'train_runtime': 17359.1112, 'train_samples_per_second': 7.712, 'train_steps_per_second': 0.772, 'total_flos': 1.4506766666563584e+17, 'train_loss': 0.12384197865515502, 'epoch': 20.0})

In [6]:
from transformers import AutoModel, AutoTokenizer, MBartForConditionalGeneration, MBartTokenizer

# โหลดโมเดลและโทเคนไนเซอร์
saved_model_path = "/content/drive/MyDrive/ABSA_Datasets/Model/test_model_attention_mask/save_model"  # แก้ไขเป็น path ของคุณ
model = MBartForConditionalGeneration.from_pretrained(saved_model_path)
tokenizer = MBartTokenizer.from_pretrained(saved_model_path)

# Save Model แบบทั่วไป

In [ ]:
# Save the model and tokenizer
saved_model_path = os.path.join(model_path, 'save_model')
model.save_pretrained(saved_model_path)
tokenizer.save_pretrained(saved_model_path)

('/content/drive/MyDrive/ABSA_Datasets/Model/test_model_attention_mask/save_model/tokenizer_config.json',
 '/content/drive/MyDrive/ABSA_Datasets/Model/test_model_attention_mask/save_model/special_tokens_map.json',
 '/content/drive/MyDrive/ABSA_Datasets/Model/test_model_attention_mask/save_model/sentencepiece.bpe.model',
 '/content/drive/MyDrive/ABSA_Datasets/Model/test_model_attention_mask/save_model/added_tokens.json')

# **Save Model แบบ State dict**

In [8]:
# บันทึกโมเดลเป็น .pth
torch.save(model.state_dict(), "/content/drive/MyDrive/ABSA_Datasets/Model/ABSA_model_state/model.pth")

# บันทึก tokenizer เป็น JSON หรือ pickle (เลือกใช้)
tokenizer.save_pretrained("/content/drive/MyDrive/ABSA_Datasets/Model/ABSA_model_state/tokenizer")

('/content/drive/MyDrive/ABSA_Datasets/Model/ABSA_model_state/tokenizer/tokenizer_config.json',
 '/content/drive/MyDrive/ABSA_Datasets/Model/ABSA_model_state/tokenizer/special_tokens_map.json',
 '/content/drive/MyDrive/ABSA_Datasets/Model/ABSA_model_state/tokenizer/sentencepiece.bpe.model',
 '/content/drive/MyDrive/ABSA_Datasets/Model/ABSA_model_state/tokenizer/added_tokens.json')

# **Save Model แบบ pth**

In [ ]:
# บันทึกโมเดลเป็น .pth
torch.save(model, "/content/drive/MyDrive/ABSA_Datasets/Model/ABSA_model_pth/model.pth")

# บันทึก tokenizer เป็น JSON หรือ pickle (เลือกใช้)
tokenizer.save_pretrained("/content/drive/MyDrive/ABSA_Datasets/Model/ABSA_model_pth/tokenizer")

# **ทดสอบ Model**

In [9]:
def generate_text(input_text):
    input_ids = tokenizer(input_text, return_tensors="pt").input_ids

    device = torch.device("cuda")

    input_ids = input_ids.to(device)
    model.to(device)

    outputs = model.generate(input_ids, num_beams=3, do_sample=False, max_length=50)
    return tokenizer.decode(outputs[0])

# Example
input_text = "จับตา OR ยอดใช้น้ำมันพุ่ง – Amazon มาแรง"
print(generate_text(input_text))

</s> <SYMBOL> OR <ASPECT> ยอดใช้น้ํามัน <OPINION> พุ่ง <POS> </s>
